# **AER850 Project 3 - YOLOv11**

# *STEP 2*

For this stage you are to use Ultralytics to train a model to detect components on a PCB. It is recommended for you to use YOLOv11 nano as your pretrained model. From this model, fine-tuning can be done to cater the model to our use case. Since this model is pretrained, no modifications to the structure of the neural network are required. However, due to the size of the dataset used for this training, the time for training will be much longer than that of previous projects.

 • Using the dataset provided, train a YOLOv11 model using Ultralytics’ model.train() tool. The dataset is already formatted to be used with YOLOv11. Three key hyperparameters you may want to include are:

– epochs: defines the number of iterations to be run. With regard to epochs, remain below 200.

– batch: defines the number of images to be analyzed in each batch and divides the number of images in the training dataset by batch size to determine the number of batches required.

– imgsz: defines the image size to be used (takes a single value, e.g. 640, 800, or 1200). The minimum recommended size is 900, as small components must be recognized.

– name: may also be included to define the name of the model to be trained.

In [1]:
!pip install ultralytics opencv-python matplotlib pillow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00


In [2]:
# Import Libraries
from ultralytics import YOLO
from google.colab import drive
import cv2
import numpy as np
import os
import torch
drive.mount('/content/drive')

# Check for GPU Usage
print("CUDA available:", torch.cuda.is_available())

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive
CUDA available: True


In [3]:
# Define YOLO Model
model = YOLO("yolo11n.pt")

# Define YAML path
DATA_YAML = "/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/data.yaml"

# Define hyperparameters
IMG_SIZE = 1088
BATCH    = 16
EPOCHS   = 170
PATIENCE = 25
MODEL_NAME = "aer850_sp_pj3_yolo11n"

# Initiate training
results = model.train(
    data=DATA_YAML,
    imgsz=IMG_SIZE,
    batch=BATCH,
    epochs=EPOCHS,
    patience=PATIENCE,
    name=MODEL_NAME,
    device=0
)

Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/Project 3 Data/data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=170, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1088, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=aer850_sp_pj3_yolo11n, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, 

# *STEP 3*
Once your model is trained, you must evaluate three images found in the Evaluation folder of the dataset. You will need to use the model.predict() function from Ultralytics to test each image. Some components may not be detected or may be mislabeled based on how many epochs were run. In terms of accuracy your evaluations should fall between the following two runs:

- Low Epoch Evaluation Results
- High Epoch Evaluation Results

In [ ]:
# Import Step 3 libraries
from PIL import Image
from IPython.display import display

# Load best.pt model to YOLO
model = YOLO("/content/drive/MyDrive/Colab Notebooks/AER850 Project 3/best.pt")

# Define evaluation and save directories
eval_dir = "/content/drive/MyDrive/Colab Notebooks/AER850 Project 3/Project 3 Data/data/evaluation"
save_dir = "/content/drive/MyDrive/Colab Notebooks"

# Load images
images = {
    "Arduino_Mega": os.path.join(eval_dir, "ardmega.jpg"),
    "Arduino_Uno": os.path.join(eval_dir, "arduno.jpg"),
    "Raspberry_Pi": os.path.join(eval_dir, "rasppi.jpg")
}

# Fix large box and large text problem
# Display class names and confidence on picture
for name, img_path in images.items():
    results = model.predict(source=img_path, conf=0.4, save=False, imgsz=640)

    im = results[0].plot(
        conf=True,
        labels=True,
        font_size=16,
        line_width=3
    )

# Display Images
    im = Image.fromarray(im)
    out_path = f"{save_dir}/{name}.jpg"
    im.save(out_path)

    print(name)
    display(Image.open(out_path))